# Project 1 - Explanitory visualization 
## Information Visualization 

### Datacleaning

In [4]:
import altair as alt
import pandas as pd
import numpy as np
from vega_datasets import data

First we have to investigate the data we have been provided. Therfore, the three files are loaded using pandas, and then they are displayed as raw data first, only taking the seperator into account. 

In [5]:

#Data overview of each file
df_cruz = pd.read_csv('cruz_list.csv')
display(df_cruz.head())


,grant_number;in_cruz_list
0,2011780;FALSE
1,2027519;FALSE
2,2034824;FALSE
3,2040209;FALSE
4,2040684;FALSE


In this file, it is noticed that the columns are not seperated correctly and that the values for true and false are strings and not actually booleans. Furthermore the grant number also has to be defined as an integer. Furthermore the values in gran_number is the same as grant_id in df_nsf, so by renaming the column, it can be merged. 

In [6]:
df_flagged = pd.read_csv('flagged_words_trump_admin.csv')
display(df_flagged.head())


,flagged_word
0,activism
1,activists
2,advocacy
3,advocate
4,barrier


This file contains data of flagged words, and are not related to a key column. As it is only a file containing a column with the string of words that are flagged, it is possible to implement this data into the merged dataset as booleans. 

In [7]:
df_nsf = pd.read_csv('nsf_terminations_airtable.csv')
display(df_nsf.head())

,grant_id,status,terminated,suspended,termination_date,reinstated,reinstatement_date,reinstatement_indicator,nsf_url,usaspending_url,...,usaspending_obligated,usaspending_outlaid,estimated_budget,estimated_outlays,estimated_remaining,division,directorate,div,dir,record_sha1
0,2201796,❌ Terminated,True,False,2025-04-18,False,NaN,NaN,https://www.nsf.gov/awardsearch/showAward?AWD_...,https://www.usaspending.gov/award/ASST_NON_220...,...,1083218.0,901981.05,1083218,9.019811e+05,1.812369e+05,Research on Learning in Formal and Informal Se...,STEM Education,DRL,EDU,5866299f5d416a23e01823f57f857aad2950cd4d
1,2315095,❌ Terminated,True,False,2025-04-18,False,NaN,NaN,https://www.nsf.gov/awardsearch/showAward?AWD_...,https://www.usaspending.gov/award/ASST_NON_231...,...,NaN,NaN,3641116,3.641116e+06,0.000000e+00,NaN,NaN,NaN,NaN,136bbe81751eb4f4ae26f6dceb68ee1339877daa
2,2236163,❌ Terminated,True,False,2025-04-18,False,NaN,NaN,https://www.nsf.gov/awardsearch/showAward?AWD_...,https://www.usaspending.gov/award/ASST_NON_223...,...,609429.0,348712.21,609429,3.487122e+05,2.607168e+05,Undergraduate Education,STEM Education,DUE,EDU,075e2156a2996eba5595f0095b20da6da75c06e7
3,2243822,🔄 Possibly Reinstated,True,False,2025-04-18,True,2025-06-30,Thakur v. Trump (Jun 2025),https://www.nsf.gov/awardsearch/showAward?AWD_...,https://www.usaspending.gov/award/ASST_NON_224...,...,NaN,NaN,3000000,1.079409e+06,1.920591e+06,NaN,NaN,NaN,NaN,f9a85b61f807953ecdb2a77c92362c042dcfb494
4,2216648,❌ Terminated,True,False,2025-04-18,False,NaN,NaN,https://www.nsf.gov/awardsearch/showAward?AWD_...,https://www.usaspending.gov/award/ASST_NON_221...,...,2719700.0,1547300.07,2719700,1.547300e+06,1.172400e+06,Biological Infrastructure,Biological Sciences,DBI,BIO,f35dec9f84e3160f0ff2eee79200f25ff5471f67


The first thing that pops into the eye is the status column,  which could be made into a binary value, and that way not disurb the eye as much as the raw data and it could also be handy in the further analysis. Furthermore there are 35 columns, and by looking at the question in the project, it is not nescessary to keep them all. Therfore an array with the relevant comlums are made. It is also recognised that there is a shared column with the fist datafile containing the grant_id, and therefore the two can be merged. Below are the commands described for each datafile done. 

In [8]:
#data treatment for cruz_list.csv
df_cruz[['grant_number', 'in_cruz_list']] = df_cruz['grant_number;in_cruz_list'].str.split(';', expand=True)
df_cruz['grant_id'] = df_cruz['grant_number'].astype(int)
df_cruz['in_cruz_list'] = df_cruz['in_cruz_list'].map({'TRUE': True, 'FALSE': False})


#Data treatment for nsf.csv
df_nsf['status'] = df_nsf['status'].map({'❌ Terminated': 0, '🔄 Possibly Reinstated': 1})

keep = [
    'grant_id',                  # key column
    'org_name',                  # institution name
    'org_state',                 # state for Q1
    'org_city',                  # (optional) for detailed visuals
    'project_title', 'abstract', # for flagged words analysis
    'nsf_total_budget',          # main budget field (Q3)
    'status',                    # cancelled vs reinstated
    'termination_date',          # when it was terminated
    'reinstated',                # boolean
    'reinstatement_date'         # date
]

df_nsf_filtered = df_nsf[[c for c in keep if c in df_nsf.columns]].copy()

df_merged = pd.merge(df_nsf_filtered, df_cruz[['grant_id', 'in_cruz_list']], on='grant_id', how='left')

# flagged-word detection
import re

flagged_words = df_flagged['flagged_word'].dropna().str.lower().tolist()
# avoid calling the builtin name `map` (it may have been shadowed elsewhere);
# use a list comprehension to escape words safely
escaped_words = [re.escape(w) for w in flagged_words]
pattern = r'\b(' + '|'.join(escaped_words) + r')\b'

# Combine project title + abstract, search for flagged terms
df_merged['has_flagged_word'] = (
    (df_merged['project_title'].fillna('') + ' ' + df_merged['abstract'].fillna(''))
    .str.lower()
    .str.contains(pattern, regex=True, na=False)
)


display(df_merged.head())


C:\Users\matil\AppData\Local\Temp\ipykernel_11908\2236927903.py:40: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  .str.contains(pattern, regex=True, na=False)


,grant_id,org_name,org_state,org_city,project_title,abstract,nsf_total_budget,status,termination_date,reinstated,reinstatement_date,in_cruz_list,has_flagged_word
0,2201796,Carnegie-Mellon University,PA,PITTSBURGH,Collaborative Research: Investigating Gender D...,Despite evidence that gender differences in ma...,1051218,0,2025-04-18,False,NaN,True,True
1,2315095,University of Texas at San Antonio,TX,SAN ANTONIO,Rhetorical Engineering Education to Support Pr...,This project will provide a handbook of materi...,3641116,0,2025-04-18,False,NaN,True,True
2,2236163,University of New Mexico,NM,ALBUQUERQUE,"Increasing the Effectiveness of Justice, Equit...",This project aims to serve the national intere...,609429,0,2025-04-18,False,NaN,True,True
3,2243822,University of California-Berkeley,CA,BERKELEY,NRT-HDR: Computational Research for Equity in ...,The criminal legal system is an important driv...,3000000,1,2025-04-18,True,2025-06-30,True,True
4,2216648,Oklahoma State University,OK,STILLWATER,RaMP: Oklahoma Network addressing human impact...,Building a stronger and more diverse scientifi...,2719700,0,2025-04-18,False,NaN,True,True


Another step that is done in order to futher decide how to treat the initial data, is to diplay information of each file, containing the non-null count, the type of each row and the range of the data. 

In [9]:
#Data information 
print("\nCruz data info:")
print(df_merged.info())
print(df_merged.isna().sum())



Cruz data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1970 entries, 0 to 1969
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   grant_id            1970 non-null   int64 
 1   org_name            1970 non-null   object
 2   org_state           1970 non-null   object
 3   org_city            1969 non-null   object
 4   project_title       1970 non-null   object
 5   abstract            1969 non-null   object
 6   nsf_total_budget    1970 non-null   int64 
 7   status              1970 non-null   int64 
 8   termination_date    1970 non-null   object
 9   reinstated          1970 non-null   bool  
 10  reinstatement_date  420 non-null    object
 11  in_cruz_list        1039 non-null   object
 12  has_flagged_word    1970 non-null   bool  
dtypes: bool(2), int64(3), object(8)
memory usage: 173.3+ KB
None
grant_id                 0
org_name                 0
org_state                0
org_city 

In [10]:
print("\nFlagged words data info:")
print(df_flagged.info())
print(df_flagged.isna().sum())



Flagged words data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   flagged_word  54 non-null     object
dtypes: object(1)
memory usage: 564.0+ bytes
None
flagged_word    0
dtype: int64


### Question 1: How are the cancellations distributed by states? 

In [11]:
# Count terminations by state
state_counts = (
    df_merged[df_merged['status'] == 0]['org_state']
    .value_counts()
    .reset_index()
)
state_counts.columns = ['state', 'Terminations']

# Select top 10 states
#top10_states = state_counts.head(10)


alt.Chart(state_counts).mark_bar(color='steelblue').encode(
    y=alt.Y('state:N', sort='-x', title='state'),
    x=alt.X('Terminations:Q', title='Number of Cancellations')
).properties(
    title='Top 10 States With Most Cancellations'
)

alt.Chart(...)

In [12]:
#@title Top 10 States by NSF Terminations

state_counts = (
    df_merged[df_merged['status'] == 0]['org_state']
    .value_counts()
    .reset_index()
)
state_counts.columns = ['state', 'Terminations']

top10_states = state_counts.head(10)

base = alt.Chart(top10_states, title='Top 10 States by NSF Terminations').encode(
    y=alt.Y('state:N', sort='-x', title='State', axis=alt.Axis(grid=False)),
    x=alt.X('Terminations:Q', title='Terminations', axis=alt.Axis(grid=False))
)

bars = base.mark_rule(color='lightgray').encode(
    x='Terminations:Q',
    x2=alt.value(0)
)

points = base.mark_circle(color='black', size=80)

text = base.mark_text(
    align='left',
    dx=6,
    fontSize=12
).encode(
    text='Terminations:Q'
)

chart = bars + points + text

chart.configure_axis(
    grid=False
).configure_view(
    stroke=None  
)


alt.LayerChart(...)

In [13]:
# Terminated NSF Grants per state

map = alt.topo_feature(data.us_10m.url, feature='states')

usChart = alt.Chart(map).mark_geoshape(
    fill='lightgray',
    stroke='white',
).properties(
    width=600,
    height=360
).project('albersUsa')

df = (
    df_merged[df_merged['status'] == 0]
    .groupby('org_state')['grant_id']
    .nunique()
    .reset_index(name='Terminations')
)

abbr2name = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
    'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
    'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
    'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota','MS':'Mississippi',
    'MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey',
    'NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
    'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
    'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming','DC':'District of Columbia'
}
df['state_name'] = df['org_state'].map(abbr2name)

capitals = data.us_state_capitals.url

terminationsMap = alt.Chart(df, title='Terminated NSF Grants distributed by State'
).transform_lookup(
    lookup='state_name',
    from_=alt.LookupData(capitals, key='state', fields=['state','lat','lon'])
).transform_filter(
    alt.datum.lat != None
).mark_circle(opacity=0.8).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    size=alt.Size('Terminations:Q', title='Terminations', scale=alt.Scale(type='sqrt', range=[10, 1000])),
    color=alt.Color('Terminations:Q', title='Terminations', scale=alt.Scale(scheme='reds')),
    tooltip=['state_name:N','org_state:N','Terminations:Q']
)

usChart + terminationsMap


alt.LayerChart(...)

In [14]:

map = alt.topo_feature(data.us_10m.url, feature='states')

usChart = alt.Chart(map).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=600,
    height=360
).project('albersUsa')

df = (
    df_merged[df_merged['status'] == 0]
    .groupby('org_state')['grant_id']
    .nunique()
    .reset_index(name='Terminations')
)

abbr2name = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
    'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
    'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
    'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota','MS':'Mississippi',
    'MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey',
    'NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
    'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
    'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming','DC':'District of Columbia'
}
df['state_name'] = df['org_state'].map(abbr2name)

capitals = data.us_state_capitals.url

terminationsMap = (
    alt.Chart(df, title='Terminated NSF Grants distributed by State')
    .transform_lookup(
        lookup='state_name',
        from_=alt.LookupData(capitals, key='state', fields=['state', 'lat', 'lon'])
    )
    .transform_filter(
        alt.datum.lat != None
    )
    .mark_circle(size=120, opacity=0.85, stroke='white', strokeWidth=0.4)
    .encode(
        longitude='lon:Q',
        latitude='lat:Q',
        color=alt.Color('Terminations:Q',
                        title='Terminations',
                        scale=alt.Scale(scheme='reds')),
        tooltip=['state_name:N', 'org_state:N', 'Terminations:Q']
    )
)

(usChart + terminationsMap).configure_view(stroke=None)


alt.LayerChart(...)

For the first question, we started by making a very trivial bar chart containing all states and their values. By doing so, the chart can be said to follow the principle of expressiveness, as it displays all the information from the attributes without adding or removing anything. However, it is not easy for the viewer to comprehend the distribution, as there are too many bars, which can make comparison difficult. Therefore, we decided to filter the data by displaying the 10 states with the highest number of terminations, and furthermore, we changed the chart type to a lollipop chart. This made the view very minimalistic and easy to read, but as the question is about the distribution, we concluded that we had to show all the states. Therefore, our next idea was a choropleth, as it combines the viewer’s prior knowledge of the placement of states with the ability to display all the terminations of the states. The first choropleth also varied the magnitude of the points to emphasize the number of terminations. This feature made some of the values overlap and thereby made the chart look cluttered. Therefore, the final chart shows the increasing number of terminations by a color intensity label only. By making these choices, it was possible to capture all the states and show the distribution of terminations by color intensity in one chart, without causing unnecessary clutter.Lastly, by choosing colorgrading of the same red color, we avoid fx red-green conflict with color-blindness. This color is used througout the charts, insicating the negatively loaded attributes. 


### Question 2:  What are the institutions that have been more affected in terms of number of cancelled grants. How does this compare to the others? 

In [15]:
# Count cancelled grants by institution name
institution_counts = (
    df_merged[df_merged['status'] == 0]['org_name']
    .value_counts()
    .reset_index()
)
institution_counts.columns = ['Institution', 'Cancelled Grants']

# Select top 10 institutions with most cancellations
top10_institutions = institution_counts.head(10)

# Visualize
alt.Chart(top10_institutions).mark_bar(color='indianred').encode(
    y=alt.Y('Institution:N', sort='-x', title='Institution'),
    x=alt.X('Cancelled Grants:Q', title='Number of Cancelled Grants'),
    tooltip=['Institution:N', 'Cancelled Grants:Q']
).properties(
    title='Top 10 Institutions With Most Cancelled Grants',
    width=300
    
)


alt.Chart(...)

A horizontal bar chart is used to answer this question. It's used because is good at compare categories with long labels and sorting by descending cancellations makes it easy to get an overview of the most affected institutions. Not all institutions are visible to keep labels readable and avoid a long tail of bars. The same red color is used for all bars for not overwhelming the labels. 

### Question 3: What are the institutions that have been more affected in terms of budget, and how does this compare to the others? 

In [16]:
# Selecting data
q3_cancel = df_merged.loc[df_merged['status'] == 0, ['org_name', 'nsf_total_budget']]

q3_agg = (
    q3_cancel
    .groupby('org_name', as_index=False)
    .agg(
        total_cancelled_budget=('nsf_total_budget', 'sum'),
        n_cancel=('nsf_total_budget', 'size')
    )
)

# Scatter plot — smaller, cleaner dots
chart_q3_scatter = (
    alt.Chart(q3_agg)
    .mark_circle(size=60, opacity=0.8, color='indianred')  # smaller dots
    .encode(
        x=alt.X('n_cancel:Q', title='Cancellations'),
        y=alt.Y('total_cancelled_budget:Q', title='Total Cancelled Budget (USD)'),
        tooltip=[
            alt.Tooltip('org_name:N', title='Institution'),
            alt.Tooltip('n_cancel:Q', title='Cancellations'),
            alt.Tooltip('total_cancelled_budget:Q', title='Budget', format=',')
        ]
    )
    .properties(
        width=600,
        height=420,
        title='Budget vs. Cancellations per Institution'
    )
    .configure_axis(grid=True)
    .configure_view(stroke=None)
)

chart_q3_scatter


alt.Chart(...)

In [17]:
# Bubble chart with labels (LOG X)
TOPK = 15
q3_top = q3_agg.sort_values('total_cancelled_budget', ascending=False).head(TOPK)
base = alt.Chart(q3_top).encode(
    x=alt.X('n_cancel:Q',
            title='Cancelled Grants (log)',
            scale=alt.Scale(type='log', base=10),   # ← log scale
            axis=alt.Axis(tickCount=5)),
    y=alt.Y('total_cancelled_budget:Q', title='Total Cancelled Budget'),
)

bubbles = base.mark_circle(opacity=0.7).encode(
    size=alt.Size('total_cancelled_budget:Q', title='Total Budget',
                  scale=alt.Scale(range=[200,800])),
    color=alt.value('indianred'),
    tooltip=['org_name:N', 'n_cancel:Q',
             alt.Tooltip('total_cancelled_budget:Q', format=',')]
)

labels = base.transform_window(rank='rank()').transform_filter(
    alt.datum.rank <= 3
).mark_text(align='center', dy=-10, fontSize=12, color='black').encode(
    text='org_name:N'
)

chart_q3_bubbles_log = (bubbles + labels).properties(
    width=500, height=400,
    title='Cancelled Budget vs. Cancelled Grants (log x-axis)'
).configure_axis(grid=True).configure_view(stroke=None)

chart_q3_bubbles_log


alt.LayerChart(...)

In [18]:
# Bar chart of top 10 institutions (highest cancelled budget) — with right legend per institution

CB_PALETTE_NEUTRAL = [
    "#B55457",  # muted indian red
    "#C97D63",  # warm clay
    "#D4A373",  # sand beige
    "#B6A19E",  # stone gray
    "#9E9E9E",  # neutral gray
    "#8C7A6B",  # taupe
    "#A68072",  # terracotta neutral
    "#D2B48C",  # tan
    "#BC8A7E",  # dusty rose
    "#A67B5B",  # light brown
    "#C4B6A6",  # pale warm gray
    "#B56576",  # soft mauve
    "#8D6E63",  # warm brown gray
    "#9A8C98",  # lilac gray
    "#6D6875"   # slate mauve gray
]

tmp = df_merged.copy()
tmp['nsf_total_budget'] = pd.to_numeric(tmp['nsf_total_budget'], errors='coerce')

# Data (only cancelled grants)
inst_data = (
    tmp[tmp['status'] == 0]
    .groupby('org_name', dropna=False)
    .agg(
        BudgetCancelled=('nsf_total_budget', 'sum'),
        CancelledGrants=('nsf_total_budget', 'count')
    )
    .reset_index()
)

# Convert to millions for readability
inst_data['BudgetMillions'] = inst_data['BudgetCancelled'] / 1e6

# ✅ Top 10 institutions by highest cancelled budget
top10 = inst_data.sort_values('BudgetMillions', ascending=False).head(10).reset_index(drop=True)

# Color mapping
domain_names = top10['org_name'].tolist()
color_scale = alt.Scale(domain=domain_names, range=CB_PALETTE_NEUTRAL[:len(domain_names)])

# Base chart (y labels hidden; legend used instead)
base = alt.Chart(top10).encode(
    y=alt.Y('org_name:N', sort=domain_names, title=None, axis=None),
    x=alt.X('BudgetMillions:Q',
            title='Total Cancelled Budget (M USD)',
            axis=alt.Axis(format='.0f'))
)

# Bars with legend (no text labels)
bars = base.mark_bar().encode(
    color=alt.Color(
        'org_name:N',
        title='Institution',
        scale=color_scale,
        legend=alt.Legend(
            orient='right',
            offset=60,
            columns=1,
            symbolSize=200,
            labelFontSize=11,
            titleFontSize=12,
            labelLimit=250
        )
    ),
    tooltip=[
        alt.Tooltip('org_name:N', title='Institution'),
        alt.Tooltip('BudgetMillions:Q', title='Total Cancelled Budget (M USD)', format='.1f'),
        alt.Tooltip('CancelledGrants:Q', title='Number of Terminated Grants')
    ]
)

# ✅ Final chart (no text on bars)
chart_inst_legend = bars.properties(
    width=700,
    height={'step': 20},
    title='Top 10 Institutions by Highest Cancelled Budget'
).configure_axis(
    grid=False
).configure_view(
    stroke=None
)

chart_inst_legend


alt.Chart(...)

In [19]:
# Bar chart of top 10 institutions by total cancelled budget (status = 0)
tmp = df_merged.copy()
tmp['nsf_total_budget'] = pd.to_numeric(tmp['nsf_total_budget'], errors='coerce')

# Data
inst_data = (
    tmp[tmp['status'] == 0]
    .groupby('org_name', dropna=False)
    .agg(
        BudgetCancelled=('nsf_total_budget', 'sum'),
        CancelledGrants=('nsf_total_budget', 'count')
    )
    .reset_index()
)

#  x-axis formatting 
inst_data['BudgetMillions'] = inst_data['BudgetCancelled'] / 1e6

# top 10 institutions with the highest total cancelled budget
top10 = inst_data.sort_values('BudgetMillions', ascending=False).head(10)

# Base chart
base = alt.Chart(top10).encode(
    y=alt.Y(
        'org_name:N',
        sort=top10['org_name'].tolist(),
        title='Institution'
    ),
    x=alt.X(
        'BudgetMillions:Q',
        title='Total Cancelled Budget (M USD)',
        axis=alt.Axis(format='.0f')
    )
)

# Bars 
bars = base.mark_bar(color='indianred').encode(
    tooltip=[
        alt.Tooltip('org_name:N', title='Institution'),
        alt.Tooltip('BudgetMillions:Q', title='Total Cancelled Budget (M USD)', format='.1f'),
        alt.Tooltip('CancelledGrants:Q', title='Number of Terminated Grants')
    ]
)

# Final chart
chart_inst = bars.properties(
    width=550,
    height=420,
    title='Top 10 Institutions Most Affected by Cancelled Budget'
).configure_axis(
    grid=False
).configure_view(
    stroke=None
)

chart_inst


alt.Chart(...)

In [20]:
# Bar chart of top 10 institutions by total cancelled budget (status = 0)
tmp = df_merged.copy()
tmp['nsf_total_budget'] = pd.to_numeric(tmp['nsf_total_budget'], errors='coerce')

# data
inst_data = (
    tmp[tmp['status'] == 0]
    .groupby('org_name', dropna=False)
    .agg(
        BudgetCancelled=('nsf_total_budget', 'sum'),
        CancelledGrants=('nsf_total_budget', 'count')
    )
    .reset_index()
)

# Convert budget to millions
inst_data['BudgetMillions'] = inst_data['BudgetCancelled'] / 1e6

# top 10 institutions with highest total cancelled budget
top10 = inst_data.sort_values('BudgetMillions', ascending=False).head(10)

# Base chart
base = alt.Chart(top10).encode(
    y=alt.Y(
        'org_name:N',
        sort=top10['org_name'].tolist(),
        title='Institution'
    ),
    x=alt.X(
        'BudgetMillions:Q',
        title='Total Cancelled Budget (M USD)',
        axis=alt.Axis(format='.0f')
    )
)

# Bars (M USD)
bars = base.mark_bar(color='indianred').encode(
    tooltip=[
        alt.Tooltip('org_name:N', title='Institution'),
        alt.Tooltip('BudgetMillions:Q', title='Total Cancelled Budget (M USD)', format='.1f'),
        alt.Tooltip('CancelledGrants:Q', title='Number of Terminated Grants')
    ]
)

# Labels: number of terminated grants
labels = base.mark_text(
    align='left',
    dx=5,
    fontSize=11,
    color='black'
).transform_calculate(
    label_text='format(datum.CancelledGrants, ".0f") + " grants"'
).encode(text='label_text:N')

# Combine and style
chart_inst = (bars + labels).properties(
    width=550,
    height=420,
    title='Top 10 Institutions Most Affected by Cancelled Budget'
).configure_axis(
    grid=False
).configure_view(
    stroke=None
)

chart_inst


alt.LayerChart(...)

The aim of this question is to identify which institutions have been most affected in terms of budget. Since the term “affected” refers to institutions that experienced cancelled grants, the data was filtered to include only entries where status = 0. This ensures that the visualization reflects the cancelled budgets rather than all grants.

The initial idea was to use a scatterplot to show the relationship between each institution’s total cancelled budget and the number of terminated grants. However, many institutions had similar values, which made it difficult to visually distinguish them. To improve readability, a logarithmic x-axis was tested, which slightly increased separation but reduced the intuitive understanding of the data.

Therefore, we decided to use a bar chart instead, as it provides a clearer and more direct comparison between institutions. Initially, each institution was given a unique color from a large palette. Yet, since we limited the visualization to the 10 most affected institutions, this approach added unnecessary visual complexity without improving interpretability.

The final design displays the total cancelled budget (in million USD) on the x-axis, sorted in descending order. This makes it easy for the viewer to immediately identify which institutions were most financially affected. Additionally, by including the number of terminated grants in the tooltip, the chart also allows the viewer to distinguish whether an institution’s total loss comes from a few large cancelled grants or many smaller ones.

This final visualization is therefore both expressive and effective: it conveys the key message clearly while maintaining simplicity and readability.

### Question 4: Is there any correlation between the cancelled grants and the list of flagged words? 

In [21]:
#Termination rate by flagged words 

# Data prep
df_rate = (
    df_merged.assign(
        Flagged=np.where(df_merged['has_flagged_word'].astype(bool),
                         'Flagged', 'Not flagged'),
        Cancelled=(df_merged['status'] == 0).astype(int)
    )
    .groupby('Flagged', as_index=False)
    .agg(CancelRate=('Cancelled', 'mean'))  # proportion terminated
)

#chart
alt.Chart(df_rate, title='Termination Rate by Flagged Words').mark_bar(size=80).encode(
    x=alt.X('Flagged:N', title=None, axis=alt.Axis(labelFontSize=13,labelAngle=0)),
    y=alt.Y('CancelRate:Q', title='Share of grants terminated',
            axis=alt.Axis(format='.0%', labelFontSize=12)),
    color=alt.Color('Flagged:N',
                    title='Flagged word',
                    scale=alt.Scale(domain=['Flagged','Not flagged'],
                                    range=['indianred','#B0B0B0'])),  # colorblind-safe
    tooltip=[alt.Tooltip('CancelRate:Q', format='.1%')]
).properties(
    width=250,
    height=300
).configure_view(
    stroke=None
).configure_axis(
    grid=True
)


alt.Chart(...)

In [22]:
#Termination rate by flagged words - ver2
#data prep
df_rate = (
    df_merged.assign(
        Flagged=np.where(df_merged['has_flagged_word'].astype(bool), 'Flagged', 'Not flagged'),
        Cancelled=(df_merged['status'] == 0).astype(int)
    )
    .groupby('Flagged', as_index=False)
    .agg(CancelRate=('Cancelled', 'mean'))
)

bars = alt.Chart(df_rate).mark_bar(size=80).encode(
    x=alt.X('Flagged:N', title=None, axis=alt.Axis(labelAngle=0, labelFontSize=13)),
    y=alt.Y('CancelRate:Q', title='Share of Grants Terminated', axis=alt.Axis(format='.0%', labelFontSize=12)),
    color=alt.Color('Flagged:N', scale=alt.Scale(domain=['Flagged', 'Not flagged'], range=['indianred', '#B0B0B0']), legend=None),
    tooltip=[alt.Tooltip('CancelRate:Q', format='.1%')]
)

labels = alt.Chart(df_rate).mark_text(
    align='center',
    dy=-8,
    fontSize=12,
    color='black'
).encode(
    x='Flagged:N',
    y='CancelRate:Q',
    text=alt.Text('CancelRate:Q', format='.1%')
)

chart = (bars + labels).properties(
    width=280,
    height=300,
    title='Termination Rate by Flagged Words'
).configure_axis(
    grid=False,
    domain=False
).configure_view(
    stroke=None
)

chart_flagged = chart.properties(width=320, height=240)
chart_flagged

alt.LayerChart(...)

The goal of this visualization is to explore whether the presence of flagged words in project titles or abstracts is associated with a higher termination rate. Since the comparison only involves two distinct groups — Flagged and Not flagged — a simple bar chart was chosen to communicate the difference clearly and intuitively.

Initially, other visualization types such as scatterplots and stacked bars were considered. However, these introduced unnecessary complexity without improving interpretability. A two-bar layout makes it immediately clear how the two groups differ, avoiding visual clutter and focusing the reader’s attention on the relative share of terminated grants.

The y-axis represents the proportion of cancelled grants, while the x-axis distinguishes between the two groups. A clean, colorblind-safe palette was used: indian red for the “Flagged” category and a neutral gray for “Not flagged.” This contrast directs attention toward the group of analytical interest without implying causality.

Numeric labels above each bar further enhance readability, allowing the viewer to grasp precise values without relying on the axis. Finally, gridlines and legends were minimized to maintain a minimal, publication-ready aesthetic consistent with the rest of the dashboard design.

### Question 5:  Is there any correlation between the cancelled grants and the list of grants in Cruz’s list? And with respect to reinstated grants?

In [23]:
# Two rates on one axis, by Cruz group

# Data prep
rates = (
    df_merged.assign(
        Cancelled=(df_merged['status']==0).astype(int),
        Reinstated=df_merged['reinstated'].astype(int)
    )
    .groupby('in_cruz_list', as_index=False)
    .agg(cancel_rate=('Cancelled','mean'), reinstate_rate=('Reinstated','mean'))
    .assign(Group=lambda d: d['in_cruz_list'].map({True:'In Cruz list', False:'Not in Cruz list'}))
)

# Chart
base = alt.Chart(rates).encode(y=alt.Y('Group:N', title=None))
rule = base.mark_rule().encode(
    x='reinstate_rate:Q', x2='cancel_rate:Q',
    tooltip=[alt.Tooltip('cancel_rate:Q', format='.1%', title='Cancel'), alt.Tooltip('reinstate_rate:Q', format='.1%', title='Reinstate')]
    
)
# Points
left = base.mark_circle(size=100, color='#B0B0B0').encode(x='reinstate_rate:Q')
right = base.mark_circle(size=100, color='indianred').encode(x='cancel_rate:Q')

(rule + left + right).properties(
    width=520, height=140, title='Q5 · Cancellation vs Reinstatement by Cruz group'
).configure_axis(grid=False).configure_view(stroke=None)


alt.LayerChart(...)

In [24]:
# Rates relative to ALL grants in group
rates = (
    df_merged
    .groupby('in_cruz_list', as_index=False)
    .agg(
        CancelRate=('status', lambda s: (s == 0).mean()),
        ReinstateRate=('reinstated', 'mean')
    )
)
rates['Cruz'] = rates['in_cruz_list'].map({True: 'In Cruz list', False: 'Not in Cruz list'})

two_part = rates.melt(
    id_vars='Cruz',
    value_vars=['CancelRate','ReinstateRate'],
    var_name='Metric', value_name='Rate'
)
two_part['Metric'] = two_part['Metric'].map({'CancelRate':'Cancelled', 'ReinstateRate':'Reinstated'})
two_part['Order']  = two_part['Metric'].map({'Cancelled': 0, 'Reinstated': 1})

chart_q5_two_part_stack = (
    alt.Chart(two_part, title='Cancelled + Reinstated (stacked over all grants)')
    .mark_bar()
    .encode(
        x=alt.X('Cruz:N', title=None, sort=['Not in Cruz list','In Cruz list'], axis=alt.Axis(labelAngle=0)),
        y=alt.Y('Rate:Q', axis=alt.Axis(format='.0%'), title='Share of grants'),
        color=alt.Color('Metric:N',
                        scale=alt.Scale(domain=['Cancelled','Reinstated'],
                                        range=['indianred', '#B0B0B0']),
                        title=None),
        order=alt.Order('Order:Q'),
        tooltip=['Cruz:N','Metric:N', alt.Tooltip('Rate:Q', format='.1%')]
    )
    .properties(width=300, height=340)
    .configure_axis(grid=True)
    .configure_view(stroke=None)
)
chart_q5_stack = chart_q5_two_part_stack.properties(width=320, height=220)
chart_q5_stack


alt.Chart(...)

In [25]:
# Rates relative to ALL grants in group
rates = (
    df_merged
    .groupby('in_cruz_list', as_index=False)
    .agg(
        CancelRate=('status', lambda s: (s == 0).mean()),
        ReinstateRate=('reinstated', 'mean')
    )
)
rates['Cruz'] = rates['in_cruz_list'].map({True: 'In Cruz list', False: 'Not in Cruz list'})

two_part = rates.melt(
    id_vars='Cruz',
    value_vars=['CancelRate', 'ReinstateRate'],
    var_name='Metric', value_name='Rate'
)
two_part['Metric'] = two_part['Metric'].map({'CancelRate': 'Cancelled', 'ReinstateRate': 'Reinstated'})
two_part['Order'] = two_part['Metric'].map({'Cancelled': 0, 'Reinstated': 1})

chart_q5_two_part_stack = (
    alt.Chart(two_part, title='Cancelled + Reinstated (stacked over all grants)')
    .mark_bar()
    .encode(
        x=alt.X(
            'Cruz:N',
            title=None,
            sort=['Not in Cruz list', 'In Cruz list'],
            axis=alt.Axis(labelAngle=0, grid=False)
        ),
        y=alt.Y(
            'Rate:Q',
            axis=alt.Axis(format='.0%', grid=False),
            title='Share of grants'
        ),
        color=alt.Color(
            'Metric:N',
            scale=alt.Scale(domain=['Cancelled', 'Reinstated'],
                            range=['indianred', '#B0B0B0']),
            title=None
        ),
        order=alt.Order('Order:Q'),
        tooltip=['Cruz:N', 'Metric:N', alt.Tooltip('Rate:Q', format='.1%')]
    )
    .properties(width=300, height=340)
    .configure_view(stroke=None)
)

chart_q5_stack = chart_q5_two_part_stack.properties(width=320, height=220)
chart_q5_stack


alt.Chart(...)

For this question, we wanted to explore whether being on Cruz’s list affected the likelihood of a grant being cancelled or reinstated. In the first version of the chart, both outcomes were represented along a single horizontal axis, with gray circles for reinstated grants and red circles for cancelled ones. The connecting line visually linked the two outcomes for each group, providing a clear and minimalist comparison between “In Cruz list” and “Not in Cruz list.” While this layout effectively showed relational differences, it was somewhat abstract and less intuitive when comparing the proportions relative to all grants.

Then we tried to visualise it by using a stacked bar chart, where each bar represents the total share of grants within a group, subdivided into Cancelled and Reinstated segments. This structure makes it easier to understand how the two outcomes contribute to the overall composition of each group. However, because the difference between the two percentages was relatively small, we discussed whether to apply one of the chart types mentioned in the course homework on truncation of the y-axis. Our key takeaway from the paper was to be cautious with bar charts that do not start at zero, as they can easily mislead the reader. Therefore, we decided to preserve the full axis scale and include tooltips to display the exact values instead.. The color scheme was kept consistent, indian red for cancellations and neutral gray for reinstatements, to maintain coherence across the dashboard and ensure colorblind accessibility. Gridlines were removed as they did'nt serve as a help and also to maintain a clean look. 

### Final visualisation

The final visualisation keeps the color consitancy througout the plots. For a more diverse and visual pleasant for the eye, the chart for answering question 1 is made into the initial chart design of a choropleth chart. This was decided for making the visualisation more harmonic and diverse. We also have to consider that we want the attention of the user, and that would not have been the case if showing four similar charts. 

In [43]:
# ---------- LAYOUT: left = tall chart, right = two small + centered map ----------
TOTAL_W = 1300                     # samlet bredde ~ 15"
L_RATIO, R_RATIO = 1.05, 2.35
LEFT_W  = int(TOTAL_W * L_RATIO / (L_RATIO + R_RATIO))
RIGHT_W = TOTAL_W - LEFT_W

FLAG_H  = 360                     # tall left chart
SMALL_H = 230                      # small top-right charts
MAP_H   = 320                      # map below the two top-right charts

# del bredde i top-rækken: 1.7 : 1.0
top1_w = int(RIGHT_W * 1.7 / (1.7 + 1.0))
top2_w = RIGHT_W - top1_w

# map-rækken: 0.07 : 0.86 : 0.07 (spacer, map, spacer)
spacer_w = int(RIGHT_W * 0.07)
center_w = RIGHT_W - 2 * spacer_w


def blank(width, height):
    # usynlig spacer
    return alt.Chart(pd.DataFrame({'x':[0]})).mark_rect(opacity=0).properties(width=width, height=height)

# Sørg for at delgraferne har faste størrelser (undgå step-højde som kan bryde layout)
left_panel   = chart_flag.properties(width=LEFT_W,  height=FLAG_H)
top_right    = alt.hconcat(
    chart_inst.properties(width=top1_w, height=SMALL_H),
    chart_cruz.properties(width=top2_w, height=SMALL_H),
    spacing=8
)
map_centered = alt.hconcat(
    blank(spacer_w, MAP_H),
    chart_states.properties(width=center_w, height=MAP_H),
    blank(spacer_w, MAP_H),
    spacing=0
)
right_panel  = alt.vconcat(top_right, map_centered, spacing=10)

dashboard = alt.hconcat(
    left_panel,
    right_panel,
    spacing=12
).properties(
    title=alt.TitleParams(
        text="NSF Terminations Overview",
        subtitle="Flagged words · Institutions · Cruz list · State distribution (Q1–Q4)",
        anchor="start", fontSize=18, subtitleFontSize=13, offset=8
    )
).resolve_scale(color='independent') \
 .configure_axis(grid=False, domain=False) \
 .configure_view(stroke=None) \
 .configure_title(anchor='start')

dashboard


alt.HConcatChart(...)

In [ ]:
# === JUPYTER SINGLE-CELL DASHBOARD (Altair) ===
# Layout-metode: faste width+height på ALLE delgrafer + "spacer"-charts til at centrere kortet.
# Krav: df_merged skal eksistere med kolonnerne bl.a.:
# grant_id, status (0=terminated, 1=other), nsf_total_budget, reinstated, has_flagged_word, org_name, org_state

import altair as alt
import numpy as np
import pandas as pd
from vega_datasets import data

# ---------- Tema & konstanter ----------
PALETTE_RED  = 'indianred'
PALETTE_GREY = '#B0B0B0'

alt.themes.enable('none')
alt.data_transformers.disable_max_rows()

# ---------- Layoutmål (optimeret til ~13" skærm) ----------
TOTAL_W = 700                   # samlet bredde
L_RATIO, R_RATIO = 1.05, 2.35
LEFT_W  = int(TOTAL_W * L_RATIO / (L_RATIO + R_RATIO))
RIGHT_W = TOTAL_W - LEFT_W

FLAG_H  = 600
SMALL_H = 300
MAP_H   = 300

# Højre øverste række — lidt smallere plots end før (1.55 : 0.95)
top1_w = int(RIGHT_W * 1.55 / (1.55 + 0.95))
top2_w = RIGHT_W - top1_w

# Kort-række centreret med lidt mindre margin (0.05 : 0.90 : 0.05)
spacer_w = int(RIGHT_W * 0.05)
center_w = RIGHT_W - 2 * spacer_w

# Spacer-helper
def _blank(width, height):
    return alt.Chart(pd.DataFrame({'x':[0]})).mark_rect(opacity=0).properties(width=width, height=height)

# =========================
# Q3 — Termination rate by flagged words (venstre)
# =========================
if 'has_flagged_word' not in df_merged.columns:
    df_merged['has_flagged_word'] = False  # fallback hvis kolonnen ikke er beregnet endnu

df_rate = (
    df_merged.assign(
        Flagged=np.where(df_merged['has_flagged_word'].astype(bool), 'Flagged', 'Not flagged'),
        Cancelled=(df_merged['status'] == 0).astype(int)
    )
    .groupby('Flagged', as_index=False)
    .agg(CancelRate=('Cancelled', 'mean'))
)

chart_flag = (
    alt.Chart(df_rate)
    .mark_bar(size=60)
    .encode(
        x=alt.X('Flagged:N', title=None, axis=alt.Axis(labelAngle=0, labelFontSize=12, grid=False)),
        y=alt.Y('CancelRate:Q', title='Share of grants terminated',
                axis=alt.Axis(format='.0%', labelFontSize=11, grid=False)),
        color=alt.Color('Flagged:N',
                        scale=alt.Scale(domain=['Flagged','Not flagged'], range=[PALETTE_RED, PALETTE_GREY]),
                        legend=None),
        tooltip=[alt.Tooltip('CancelRate:Q', format='.1%')]
    )
    + alt.Chart(df_rate).mark_text(align='center', dy=-8, fontSize=11, color='black')
      .encode(x='Flagged:N', y='CancelRate:Q', text=alt.Text('CancelRate:Q', format='.1%'))
).properties(width=LEFT_W, height=FLAG_H)

# =========================
# Q2 — Top institutions by total cancelled budget (M USD)
# =========================
_tmp = df_merged.copy()
_tmp['nsf_total_budget'] = pd.to_numeric(_tmp['nsf_total_budget'], errors='coerce')

inst_data = (
    _tmp[_tmp['status'] == 0]
    .groupby('org_name', dropna=False)
    .agg(BudgetCancelled=('nsf_total_budget', 'sum'),
         CancelledGrants=('nsf_total_budget', 'count'))
    .reset_index()
)
inst_data['BudgetMillions'] = inst_data['BudgetCancelled'] / 1e6

top10 = inst_data.sort_values('BudgetMillions', ascending=False).head(10).copy()
domain_names = top10['org_name'].tolist()
xmax = float(top10['BudgetMillions'].max()) if len(top10) else 1.0

base_inst = alt.Chart(top10).encode(
    y=alt.Y('org_name:N', sort=domain_names, title=None, axis=alt.Axis(labelLimit=0)),
    x=alt.X('BudgetMillions:Q',
            title='Total Cancelled Budget (M USD)',
            axis=alt.Axis(labelFontSize=11, grid=False),
            scale=alt.Scale(domain=[0, xmax * 1.08]))
)

bars_inst = base_inst.mark_bar(color=PALETTE_RED).encode(
    tooltip=[
        alt.Tooltip('org_name:N', title='Institution'),
        alt.Tooltip('BudgetMillions:Q', title='Total Cancelled Budget (M USD)', format='.1f'),
        alt.Tooltip('CancelledGrants:Q', title='Number of Terminated Grants')
    ]
)

labels_inst = base_inst.mark_text(
    align='left', dx=4, baseline='middle', fontSize=11, color='black'
).transform_calculate(
    label_text='format(datum.CancelledGrants, ".0f") + " grants"'
).encode(text='label_text:N')

chart_inst = (bars_inst + labels_inst).properties(width=top1_w, height=SMALL_H)

# =========================
# Q4 — Cancelled + Reinstated (stacked) vs. Cruz-listen
# =========================
_tmp_q4 = df_merged.copy()
_tmp_q4['reinstated_num'] = pd.to_numeric(_tmp_q4['reinstated'], errors='coerce').fillna(0).astype(int)

rates = (
    _tmp_q4.groupby('in_cruz_list', as_index=False)
    .agg(
        CancelRate=('status', lambda s: (s == 0).mean()),
        ReinstateRate=('reinstated_num', 'mean')
    )
)
rates['Cruz'] = rates['in_cruz_list'].map({True: 'In Cruz list', False: 'Not in Cruz list'})

two_part = rates.melt(
    id_vars='Cruz',
    value_vars=['CancelRate', 'ReinstateRate'],
    var_name='Metric', value_name='Rate'
)
two_part['Metric'] = two_part['Metric'].map({'CancelRate': 'Cancelled', 'ReinstateRate': 'Reinstated'})
two_part['Order']  = two_part['Metric'].map({'Cancelled': 0, 'Reinstated': 1})

chart_cruz = (
    alt.Chart(two_part)
    .mark_bar()
    .encode(
        x=alt.X('Cruz:N', title=None, sort=['Not in Cruz list','In Cruz list'],
                axis=alt.Axis(labelAngle=0, labelFontSize=12, grid=False)),
        y=alt.Y('Rate:Q', title='Share of grants',
                axis=alt.Axis(format='.0%', labelFontSize=11, grid=False)),
        color=alt.Color('Metric:N',
                        scale=alt.Scale(domain=['Cancelled','Reinstated'],
                                        range=[PALETTE_RED, PALETTE_GREY]),
                        title=None),
        order=alt.Order('Order:Q'),
        tooltip=['Cruz:N','Metric:N', alt.Tooltip('Rate:Q', format='.1%')]
    )
).properties(width=top2_w, height=SMALL_H)

# =========================
# Q1 — USA map (terminated grants by state)
# =========================
map_shape = alt.topo_feature(data.us_10m.url, feature='states')
us_base = alt.Chart(map_shape).mark_geoshape(fill='lightgray', stroke='white').project('albersUsa')

state_counts = (
    df_merged[df_merged['status'] == 0]
    .groupby('org_state')['grant_id'].nunique().reset_index(name='Terminations')
)

abbr2name = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
    'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
    'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
    'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota','MS':'Mississippi',
    'MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey',
    'NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
    'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
    'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming','DC':'District of Columbia'
}
state_counts['state_name'] = state_counts['org_state'].map(abbr2name)
capitals = data.us_state_capitals.url

points = (
    alt.Chart(state_counts)
    .transform_lookup(lookup='state_name',
                      from_=alt.LookupData(capitals, key='state', fields=['state','lat','lon']))
    .transform_filter(alt.datum.lat != None)
    .mark_circle(size=120, opacity=0.85, stroke='white', strokeWidth=0.4)
    .encode(
        longitude='lon:Q',
        latitude='lat:Q',
        color=alt.Color('Terminations:Q', title='Terminations', scale=alt.Scale(scheme='reds')),
        tooltip=['state_name:N','org_state:N','Terminations:Q']
    )
)

chart_states = (us_base + points).properties(width=center_w, height=MAP_H)

# =========================
# LAYOUT — venstre høj, højre to små + centreret map (med colorbar)
# =========================
# Venstre (Flagged) – smallere, samme højde
chart_flag = chart_flag.properties(width=int(LEFT_W * 0.85), height=FLAG_H)

# Øverste højre række (inst + cruz) – lidt smallere
top_right = alt.hconcat(
    chart_inst.properties(width=int(top1_w * 0.95), height=SMALL_H),
    chart_cruz.properties(width=int(top2_w * 0.95), height=SMALL_H),
    spacing=8
)

# Map centreret under top_right med symmetriske spacers
map_centered = alt.hconcat(
    _blank(int(RIGHT_W * 0.07), MAP_H),
    chart_states.properties(width=int(RIGHT_W * 0.86), height=MAP_H),
    _blank(int(RIGHT_W * 0.07), MAP_H),
    spacing=0
)

right_panel = alt.vconcat(top_right, map_centered, spacing=10)

dashboard = alt.hconcat(
    chart_flag,
    right_panel,
    spacing=16
).properties(
    title=alt.TitleParams(
        text="NSF Terminations Overview",
        subtitle="Flagged words · Institutions · Cruz list · State distribution (Q1–Q4)",
        anchor="start", fontSize=18, subtitleFontSize=13, offset=8
    )
).resolve_scale(color='independent')

# Global konfiguration
dashboard = (
    dashboard
    .configure_axis(grid=False, domain=False)
    .configure_view(stroke=None)
    .configure_title(anchor='start')
)

dashboard


alt.HConcatChart(...)

The final visualization combines all five questions into one dashboard The layout prioritizes clarity and understandability. The visualization summarizes how NSF grant terminations were distributed during Trumps administration. The layout have four different visualizations that represent different aspect of the research questions. In the lower center we have a the map of USA that shows the geographical spread of terminations. It highlight that more cancellations were executed in Massachusetts, New York, California and Texas. The color of termination is not spread out of the whole state but have the same size for all states so there is no misconception.


The institutional bar chart goes a stage deeper and look at specify institutions. This visualization reveals which specific institutions faced significant budget cuts and how many grants they received. Finally, the Cruz list (Q5) panel shows how inclusion on the senator’s list correlates with slightly higher cancellation but minimal reinstatement.

Overall, the dashboard uses a uniform red palette to represent “termination” across plots and compact grid layout to facilitate side-by-side comparison. The same color is consistent and shows a easy understand the context and what you should focus on. In the terminations Rate by flagged words, the red color symbolizes that the Trump administration was sincerely involved and it’s the flagged words that are interesting in the overall terminations context. 
The integration of Altair and Streamlit allows for an interactive and visually balanced summary that directly answers the five questions.
